In [26]:
from os import environ

# Client Setup
from google import genai

region = environ.get("CLOUD_ML_REGION", "")
project_id = environ.get("ANTHROPIC_VERTEX_PROJECT_ID", "")
client = genai.Client(project=project_id, location=region, vertexai=True)

In [27]:
# Chunk by section
import re


def chunk_by_section(document_text):
    """Chunk text based on Markdown sections (## headers)"""
    pattern = r"\n## "
    return re.split(pattern, document_text)

In [28]:
# Embedding Generation
def generate_embedding(
    chunks: list[str] | str,
) -> list[list[float]] | list[float]:
    """Generate embeddings for the given text using the specified model."""
    is_list = isinstance(chunks, list)
    input_data = chunks if is_list else [chunks]

    response = client.models.embed_content(
        model="text-embedding-005", contents=input_data
    )

    if not response.embeddings:
        raise ValueError("API returned no embeddings")

    if len(response.embeddings) != len(input_data):
        raise ValueError(
            f"Expected {len(input_data)} embeddings, got {len(response.embeddings)}"
        )

    embeddings: list[list[float]] = []
    for i, embedding in enumerate(response.embeddings):
        if embedding.values is None:
            raise ValueError(f"Embedding {i} has no values")
        embeddings.append(embedding.values)

    return embeddings if is_list else embeddings[0]

In [29]:
# VectorIndex implementation
import math
from typing import Optional, Any, List, Dict, Tuple


class VectorIndex:
    """A simple in-memory vector index for storing and searching vectors."""

    def __init__(
        self,
        distance_metric: str = "cosine",
        embedding_fn=None,
    ):
        self.vectors: List[List[float]] = []
        self.documents: List[Dict[str, Any]] = []
        self._vector_dim: Optional[int] = None
        if distance_metric not in ["cosine", "euclidean"]:
            raise ValueError("distance_metric must be 'cosine' or 'euclidean'")
        self._distance_metric = distance_metric
        self._embedding_fn = embedding_fn

    def add_document(self, document: Dict[str, Any]):
        if not self._embedding_fn:
            raise ValueError("Embedding function not provided during initialization.")
        if not isinstance(document, dict):
            raise TypeError("Document must be a dictionary.")
        if "content" not in document:
            raise ValueError("Document dictionary must contain a 'content' key.")

        content = document["content"]
        if not isinstance(content, str):
            raise TypeError("Document 'content' must be a string.")

        vector = self._embedding_fn(content)
        self.add_vector(vector=vector, document=document)

    def search(self, query: Any, k: int = 1) -> List[Tuple[Dict[str, Any], float]]:
        if not self.vectors:
            return []

        if isinstance(query, str):
            if not self._embedding_fn:
                raise ValueError("Embedding function not provided for string query.")
            query_vector = self._embedding_fn(query)
        elif isinstance(query, list) and all(
            isinstance(x, (int, float)) for x in query
        ):
            query_vector = query
        else:
            raise TypeError("Query must be either a string or a list of numbers.")

        if self._vector_dim is None:
            return []

        if len(query_vector) != self._vector_dim:
            raise ValueError(
                f"Query vector dimension mismatch. Expected {self._vector_dim}, got {len(query_vector)}"
            )

        if k <= 0:
            raise ValueError("k must be a positive integer.")

        if self._distance_metric == "cosine":
            dist_func = self._cosine_distance
        else:
            dist_func = self._euclidean_distance

        distances = []
        for i, stored_vector in enumerate(self.vectors):
            distance = dist_func(query_vector, stored_vector)
            distances.append((distance, self.documents[i]))

        distances.sort(key=lambda item: item[0])

        return [(doc, dist) for dist, doc in distances[:k]]

    def add_vector(self, vector, document: Dict[str, Any]):
        if not isinstance(vector, list) or not all(
            isinstance(x, (int, float)) for x in vector
        ):
            raise TypeError("Vector must be a list of numbers.")
        if not isinstance(document, dict):
            raise TypeError("Document must be a dictionary.")
        if "content" not in document:
            raise ValueError("Document dictionary must contain a 'content' key.")

        if not self.vectors:
            self._vector_dim = len(vector)
        elif len(vector) != self._vector_dim:
            raise ValueError(
                f"Inconsistent vector dimension. Expected {self._vector_dim}, got {len(vector)}"
            )

        self.vectors.append(list(vector))
        self.documents.append(document)

    def _euclidean_distance(self, vec1: List[float], vec2: List[float]) -> float:
        if len(vec1) != len(vec2):
            raise ValueError("Vectors must have the same dimension")
        return math.sqrt(sum((p - q) ** 2 for p, q in zip(vec1, vec2)))

    def _dot_product(self, vec1: List[float], vec2: List[float]) -> float:
        if len(vec1) != len(vec2):
            raise ValueError("Vectors must have the same dimension")
        return sum(p * q for p, q in zip(vec1, vec2))

    def _magnitude(self, vec: list[float]) -> float:
        return math.sqrt(sum(x * x for x in vec))

    def _cosine_distance(self, vec1: list[float], vec2: list[float]) -> float:
        if len(vec1) != len(vec2):
            raise ValueError("Vectors must have the same dimension")

        mag1 = self._magnitude(vec1)
        mag2 = self._magnitude(vec2)

        if mag1 == 0 and mag2 == 0:
            return 0.0
        elif mag1 == 0 or mag2 == 0:
            return 1.0

        dot_prod = self._dot_product(vec1, vec2)
        cosine_similarity = dot_prod / (mag1 * mag2)
        cosine_similarity = max(-1.0, min(1.0, cosine_similarity))

        return 1.0 - cosine_similarity

    def __len__(self) -> int:
        return len(self.vectors)

    def __repr__(self) -> str:
        has_embed_fn = "Yes" if self._embedding_fn else "No"
        return f"VectorIndex(count={len(self)}, dim={self._vector_dim}, metric='{self._distance_metric}', has_embedding_fn='{has_embed_fn}')"

In [30]:
with open("./report.md", "r") as f:
    text = f.read()

In [31]:
# 1. Chunk the text by section
chunks = chunk_by_section(text)

In [32]:
# 2. Generate embeddings for each chunk
embeddings = generate_embedding(chunks)

In [33]:
# 3. Create a vector store and add each embedding to it
vector_store = VectorIndex(distance_metric="cosine", embedding_fn=generate_embedding)
for chunk, embedding in zip(chunks, embeddings):
    document = {"content": chunk}
    vector_store.add_vector(vector=embedding, document=document)

In [34]:
# 4. Some time later, a user will ask a question. Generate an embedding for it
user_question = "What happened for INC-2023-Q4-011"
question_embedding = generate_embedding(user_question)

In [35]:
# 5. Search the store with the embedding, find the 2 most relevant chunks
results = vector_store.search(query=question_embedding, k=2)

In [36]:
results

[({'content': 'Section 2: Software Engineering - Project Phoenix Stability Enhancements\n\nThe Software Engineering division dedicated considerable effort to improving the stability and performance of the core systems underpinning Project Phoenix. Recurring issues, particularly `ERR_MEM_ALLOC_FAIL_0x8007000E` during peak loads and `TIMEOUT_QUERY_DB_0xDEADBEEF` affecting data retrieval operations, were prioritized, at a cost of INC-2023-Q4-011. Root cause analysis pointed towards inefficiencies in the primary data caching algorithm and suboptimal database indexing strategies. The deployment of a patch addressed the memory allocation error, resulting in a measured 40% reduction in critical failures under simulated stress tests during Q4 2024 (Test Case ID: INC-2023-Q4-011). Further refactoring of the query module, scheduled for the next release cycle, aims to resolve the timeout issue. These findings underscore the importance of robust testing protocols, especially given the dependencies